In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import numpy as np
import pandas as pd
import urllib.request
import os
import matplotlib.pyplot as plt
import cv2
# import torchvision.datasets as dset
# import torchvision.transforms as T
# from torch.utils.data import DataLoader
# from torch.utils.data import sampler
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F  # useful stateless functions
PROJECT_DIR = "/content/drive/Shareddrives/CNN"
os.chdir(PROJECT_DIR)

In [ ]:
table = pd.read_csv(filepath_or_buffer='./flipkart.csv', encoding="unicode_escape")
print(table.loc[0])

uniq_id                                     c2d766ca982eca8304150849735ffef9
crawl_timestamp                                    2016-03-25 22:59:23 +0000
product_url                http://www.flipkart.com/alisha-solid-women-s-c...
product_name                             Alisha Solid Women's Cycling Shorts
product_category_tree      ["Clothing >> Women's Clothing >> Lingerie, Sl...
pid                                                         SRTEH2FF9KEDEFGF
retail_price                                                             999
discounted_price                                                         379
image                      ["http://img5a.flixcart.com/image/short/u/4/a/...
is_FK_Advantage_product                                                False
description                Key Features of Alisha Solid Women's Cycling S...
product_rating                                           No rating available
overall_rating                                           No rating available

In [ ]:
# New column for pure URL list
def slicing_img_url(urlListString):
    try:
        realList = urlListString.split("\"")
        i = 0
        while i<len(realList):
            if len(realList[i])<7: # "http://"
                del realList[i]
            else:
                realList[i] = realList[i].strip()
                i += 1
        return realList
    except:
        print(urlListString)
        return []

# New column for product labels
def slicing_category_label(product_category_tree):
    labelList = product_category_tree.split(">>")
    i = 0
    while i<len(labelList):
        labelList[i] = labelList[i].strip().strip("[").strip("]").strip("\"")
        i += 1
    return labelList[0]
    # return labelList  # return this of all labels is needed
table['image_url'] = table['image'].apply(slicing_img_url)
table['labels'] = table['product_category_tree'].apply(slicing_category_label)

nan
nan
nan


In [ ]:
print(table.loc[0])

## Download images


In [ ]:
"""
os.chdir('./image')
found = set()
not_found = set()
for id_url_list_pair in table[['uniq_id', 'image_url']].iterrows():
    # filename = id_url_list_pair[1]["uniq_id"] + '.jpg'  # Use unique id as file's name.
    index = id_url_list_pair[0]
    filename = str(index) + '.jpg'   # Use flow index
    download = False
    for each_url in id_url_list_pair[1]["image_url"]:
        # print(each_url)
        try:
            urllib.request.urlretrieve(url=each_url,filename=filename)
            found.add(index)
            download = True
            break
        except:
            continue
    if not download:
        print('No image available for product', filename)
        not_found.add(index)
os.chdir('..')
"""

No image available for product 90.jpg
No image available for product 93.jpg
No image available for product 94.jpg
No image available for product 95.jpg
No image available for product 97.jpg
No image available for product 99.jpg
No image available for product 100.jpg
No image available for product 111.jpg
No image available for product 116.jpg
No image available for product 119.jpg
No image available for product 123.jpg
No image available for product 126.jpg
No image available for product 130.jpg
No image available for product 138.jpg
No image available for product 143.jpg
No image available for product 148.jpg
No image available for product 153.jpg
No image available for product 162.jpg
No image available for product 164.jpg
No image available for product 168.jpg
No image available for product 170.jpg
No image available for product 172.jpg
No image available for product 177.jpg
No image available for product 183.jpg
No image available for product 185.jpg
No image available for product 

In [ ]:
"""
print(len(not_found))
"""

1554


## Resize the images, rewrite them, and store them into ndarray

In [ ]:
"""
def img_to_square(img, side_length=256):
    h, w, d = img.shape
    WHITE = [255,255,255]
    if h > w:
        new_h = side_length
        resized = cv2.resize(img,(int(w*new_h/h),new_h),interpolation=cv2.INTER_CUBIC)
        left_border = right_border = (new_h-int(w*new_h/h))//2
        if (new_h-int(w*new_h/h))%2 != 0:
            right_border += 1
        return cv2.copyMakeBorder(resized,0,0,left_border,right_border,cv2.BORDER_CONSTANT,value=WHITE)
    elif h < w:
        new_w = side_length
        resized = cv2.resize(img,(new_w,int(h*new_w/w)),interpolation=cv2.INTER_CUBIC)
        top_border = bottom_border = (new_w-int(h*new_w/w))//2
        if (new_w-int(h*new_w/w))%2 != 0:
            bottom_border += 1
        return cv2.copyMakeBorder(resized,top_border,bottom_border,0,0,cv2.BORDER_CONSTANT,value=WHITE)
    else:
        return cv2.resize(img,(side_length,side_length),interpolation=cv2.INTER_CUBIC)
X = []
Y = []
broken_image = set()
for i in found:
    try:
        img = cv2.imread("./image/"+str(i)+".jpg")
        square = img_to_square(img, side_length=256)
        # print(square.shape)
        cv2.imwrite("./image/"+str(i)+".jpg", square, [cv2.IMWRITE_JPEG_QUALITY, 100])
        X.append(square)
        Y.append(table['labels'][i])
    except:
        print("The %sth image is broken" % i)
        broken_image.add(i)
"""

In [ ]:
X = []
Y = []
broken_image = set()
for i in found:
    try:
        img = cv2.imread("./image/"+str(i)+".jpg")
        X.append(img)
        Y.append(table['labels'][i])
    except:
        # print("The %sth image is broken" % i)
        broken_image.add(i)

In [ ]:
print(Y)

['Clothing', 'Furniture', 'Footwear', 'Clothing', 'Pet Supplies', 'Eternal Gandhi Super Series Crystal Paper Weight...', 'Clothing', 'Furniture', 'Footwear', 'Clothing', 'Footwear', 'Clothing', 'Pet Supplies', 'Clothing', 'Pens & Stationery', 'Clothing', 'Furniture', 'Footwear', 'Clothing', 'Furniture', 'Pet Supplies', 'Clothing']


In [ ]:
from sklearn.preprocessing import LabelEncoder
X = np.array(X)
X = torch.from_numpy(X)
# labels = table['labels'][:].drop(index=list(not_found)).drop(index=list(broken_image))
labelencoder = LabelEncoder()
Y = labelencoder.fit_transform(Y)
Y = torch.from_numpy(Y)
print(Y)
print('the shape of image is: ',X.shape)
print('the shape of label is: ',Y.shape)

tensor([0, 3, 2, 0, 5, 1, 0, 3, 2, 0, 2, 0, 5, 0, 4, 0, 3, 2, 0, 3, 5, 0])
the shape of image is:  torch.Size([22, 256, 256, 3])
the shape of label is:  torch.Size([22])
